<a href="https://colab.research.google.com/github/vvshyer/tensorflow2.0_learning/blob/master/text_generation_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.2
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# 1. generate vocab
# 2. build mapping char->id
# 3. data -> id_data
# 4. abcd -> bcd<eos>

# 将字符串转为set
vocab = sorted(set(text))

print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
char2idx = {char:idx for idx, char in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [6]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [7]:
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])


[18 47 56 57 58  1 15 47 58 47]
First Citi


In [8]:
def split_input_target(id_text):
    # abcde -> abcd, bcde
    return id_text[0:-1], id_text[1:]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1, 
                                 drop_remainder = True)
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])
    
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))


tf.Tensor(18, shape=(), dtype=int64) F
tf.Tensor(47, shape=(), dtype=int64) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [9]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [0]:
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)

In [12]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        keras.layers.LSTM(units = rnn_units,
                          stateful = True,
                          recurrent_initializer='glorot_uniform',
                          return_sequences = True),
        keras.layers.Dense(vocab_size)
    ])
    
    return model

model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units, 
    batch_size = batch_size)

model.summary()

W0719 07:19:33.528047 139669300365184 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f06c73abf98>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [13]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [14]:
# random sampling. 
# greedy策略; random策略.
sample_indices = tf.random.categorical(
    logits = example_batch_predictions[0], num_samples = 1)
print(sample_indices)
# (100, 65) -> (100, 1)
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[[10]
 [17]
 [17]
 [53]
 [ 4]
 [42]
 [44]
 [64]
 [ 3]
 [38]
 [42]
 [ 8]
 [22]
 [ 9]
 [ 1]
 [49]
 [27]
 [39]
 [38]
 [46]
 [24]
 [ 4]
 [62]
 [20]
 [64]
 [44]
 [12]
 [15]
 [61]
 [13]
 [ 1]
 [61]
 [46]
 [23]
 [51]
 [10]
 [12]
 [49]
 [20]
 [20]
 [19]
 [50]
 [15]
 [26]
 [16]
 [57]
 [38]
 [42]
 [28]
 [21]
 [48]
 [38]
 [56]
 [19]
 [15]
 [64]
 [23]
 [10]
 [13]
 [55]
 [34]
 [45]
 [24]
 [ 4]
 [32]
 [49]
 [50]
 [13]
 [14]
 [60]
 [ 4]
 [12]
 [ 4]
 [51]
 [63]
 [64]
 [60]
 [33]
 [42]
 [12]
 [26]
 [48]
 [33]
 [ 3]
 [16]
 [31]
 [49]
 [37]
 [11]
 [ 9]
 [ 9]
 [59]
 [40]
 [ 9]
 [31]
 [36]
 [43]
 [53]
 [19]
 [ 7]], shape=(100, 1), dtype=int64)
tf.Tensor(
[10 17 17 53  4 42 44 64  3 38 42  8 22  9  1 49 27 39 38 46 24  4 62 20
 64 44 12 15 61 13  1 61 46 23 51 10 12 49 20 20 19 50 15 26 16 57 38 42
 28 21 48 38 56 19 15 64 23 10 13 55 34 45 24  4 32 49 50 13 14 60  4 12
  4 51 63 64 60 33 42 12 26 48 33  3 16 31 49 37 11  9  9 59 40  9 31 36
 43 53 19  7], shape=(100,), dtype=int64)


In [15]:
print("Input: ", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Output: ", repr("".join(idx2char[target_example_batch[0]])))
print()
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  "ne! all men call thee fickle:\nIf thou art fickle, what dost thou with him.\nThat is renown'd for fait"

Output:  "e! all men call thee fickle:\nIf thou art fickle, what dost thou with him.\nThat is renown'd for faith"

Predictions:  ':EEo&dfz$Zd.J3 kOaZhL&xHzf?CwA whKm:?kHHGlCNDsZdPIjZrGCzK:AqVgL&TklABv&?&myzvUd?NjU$DSkY;33ub3SXeoG-'


In [0]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

In [17]:
model.compile(optimizer = 'adam', loss = loss)

example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.173719


In [19]:
output_dir = "./text_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

epochs = 100

history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 30s 174ms/step - loss: 1.6834
Epoch 2/100
172/172 [==============================] - 30s 173ms/step - loss: 1.5268
Epoch 3/100
172/172 [==============================] - 30s 173ms/step - loss: 1.4338
Epoch 4/100
172/172 [==============================] - 30s 174ms/step - loss: 1.3679
Epoch 5/100
172/172 [==============================] - 30s 173ms/step - loss: 1.3144
Epoch 6/100
172/172 [==============================] - 30s 173ms/step - loss: 1.2674
Epoch 7/100
172/172 [==============================] - 30s 174ms/step - loss: 1.2224
Epoch 8/100
172/172 [==============================] - 30s 174ms/step - loss: 1.1781
Epoch 9/100
172/172 [==============================] - 30s 173ms/step - loss: 1.1353
Epoch 10/100
172/172 [==============================] - 30s 173ms/step - loss: 1.0973
Epoch 11/100
172/172 [==============================] - 30s 174ms/step - loss: 1.0618
Epoch 12/100
172/172 [==============================] - 30s 174

In [20]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_checkpoints/ckpt_100'

In [21]:
model2 = build_model(vocab_size,
                     embedding_dim,
                     rnn_units,
                     batch_size = 1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))
# start ch sequence A,
# A -> model -> b
# A.append(b) -> B
# B(Ab) -> model ->c
# B.append(c) -> C
# C(Abc) -> model -> ..
model2.summary()

W0719 08:14:10.494503 139669300365184 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f06c2274160>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [22]:
def generate_text(model, start_string, num_generate=1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    model.reset_states()
    
    # temperature >1, random
    # temperature <1, greedy
    temperature = 0.5
    
    for _ in range(num_generate):
        # 1. model inference -> predictions
        # predictions: [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # predictions: logits -> softmax -> prob
        # softmax: e^xi
        # eg:4,2 e^4/(e^4+e^2)=0.88, e^2/(e^4+e^2)=0.12
        # eg:2,1 e^2(e^2 + e)=0.73, e/(e^2+e)=0.27
        predictions = predictions / temperature
        # predictions: [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predictions: [input_eval_len, 1]
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # s, x -> rnn -> s', y
        input_eval = tf.expand_dims([predicted_id], 0)
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "All")
        # 2. sample -> ch ->text_generated.
        # 3. undate input_eval
print(new_text)

Allk of such a coldurers;
And cannot lie with thee as you take in your chance.

ISABELLA:
O you beast!
O faith, a great matter and so great a weighty cause
Of love between your highness or the place,
That brought them in; so hot, they will be oaths
With pure poiling where it is,
And leave your brothers to report this valour
content of our officers.

Provost:
Here is the head of tears.

Clown:
He shall be there no man: some noiser is true.

DUKE VINCENTIO:
I pray you, sir, but young King Licion
Clarence doth his eyes thereof that faced repend;
And I him in them back again,
Not this metal, of the house of York
Is a strange brooch in this appetite,
Hasting to the sou and so many lives
Unto the soldiers to the judge:
If your doubler and courage most strange
In their pilot of his offence!
What strange from such cheerly shall there is slain,
And counsel may my knight, and earth that slay these fellows?

LADY CAPULET:
Hark, here long in hope,
But I flouse thee so much with the boty?

PAULINA: